In [1]:
import pixellib
import os
import cv2
import numpy as np
import pandas as pd
import pickle

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras import backend as K
from pixellib.semantic import semantic_segmentation
from pixellib.instance import instance_segmentation


In [3]:
# Задаем пути проекта

root_dir =r'G:\bulla'

file_dir = r'G:\bulla\source_image\work'
# file_dir = r'G:\bulla\test_out_clear'

# test_dir = r'G:\bulla\test_source'
test_dir = r'G:\bulla\test\test_source'
test_out_dir= r'G:\bulla\test\test_out'

out_dir =r'G:\bulla\segm'

maps_dir =r'G:\bulla\main_data\semantic_maps'
df_picle_dir =r'G:\bulla\main_data\semantic_df_full'
weights_dir = r'G:\bulla\main_data\models_weights'

ade20k_dir =r'ade20k'
mrcnn_dir = r'mrcnn'
mrcnn_average_dir = r'mrcnn_average'
pascalvoc_dir =r'pascalvoc'


In [4]:
# создаем псевдонимы часто вызываемых функций
mypath = os.path.join

In [5]:
# Определяем функцию получения текстового значения поля сегментации по коду
def text_labels(labels,df):
    temp_labels =[]
    for x in labels:
        label = df[df['Idx']==x]['Name'].item()
        temp_labels.append(label)
    return temp_labels

In [12]:
segment_image = semantic_segmentation()

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# Модель сегментации Xception_ade20k

In [13]:
# загружаем в модель таблицу весов
segment_image.load_ade20k_model(mypath(weights_dir,'deeplabv3_xception65_ade20k.h5'))

In [14]:
# загружаем таблицу цветовой маркировки ade20k в датафрейм pandas
labels_table_ade20k = mypath(maps_dir,r'ade20k_semantic_segmentation_classes.txt')
df_ade20k_labels = pd.read_csv(mypath(root_dir,labels_table_ade20k), sep='\t')

In [15]:
# Тестируем работоспособность 
test_file = r'10194039.1.jpg'
segvalues, output  = segment_image.segmentAsAde20k(mypath(test_dir,test_file),overlay = True)
cv2.imwrite(mypath(test_out_dir,test_file[:-4])+'.segm.jpg', output)
labels = list(np.unique(segvalues))
print(text_labels(labels,df_ade20k_labels))
print(output.shape)
print(str(mypath(test_out_dir,test_file[:-4])+'.segm.jpg'))


C:\Python36\lib\site-packages\tensorflow\python\keras\engine\training.py:2464: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


['wall', 'building;edifice', 'sky', 'floor;flooring', 'road;route', 'person;individual;someone;somebody;mortal;soul', 'car;auto;automobile;machine;motorcar', 'box', 'signboard;sign', 'truck;motortruck', 'dirt;track', 'van', 'plaything;toy', 'minibike;motorbike']
(609, 800, 3)
G:\bulla\test\test_out\10194039.1.segm.jpg


In [105]:
# Запускаем цикл Xception ade20k для основного набора данных. Получаем зображения с наложением маски и датафрейм  разметкой сущностей.
model_dir =ade20k_dir
df_ade20k_metadata = pd.DataFrame(columns=['ID', 'Things', 'Mask'])
for dirpath, dirnames, filenames in os.walk(file_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                segvalues, output  = segment_image.segmentAsAde20k(mypath(file_dir,x),overlay = True)
                cv2.imwrite(mypath(out_dir,model_dir,x[:-4])+'.segm.jpg', output)
                labels = text_labels(list(np.unique(segvalues)),df_ade20k_labels)
                row = {'ID':x[:-6], 'Things':labels, 'Mask':segvalues}
                df1 = pd.DataFrame([row])
                df_ade20k_metadata = pd.concat([df_ade20k_metadata,df1],axis = 0,ignore_index=True)
         

In [19]:
# Загружаем дамп метаданных ade20k
with open(mypath(r'G:\bulla\main_data\semantic_df_full',r'df_ade20k_metadata.pickle'), 'rb') as f:
    df_ade20k_metadata = pickle.load(f)

In [20]:
df_ade20k_metadata

,ID,Things,Mask
0,10193973,"[wall, floor;flooring, ceiling, cabinet, perso...","[[1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,..."
1,10193980,"[wall, floor;flooring, ceiling, person;individ...","[[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, ..."
2,10193983,"[wall, floor;flooring, ceiling, person;individ...","[[6, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
3,10194012,"[wall, floor;flooring, ceiling, person;individ...","[[6, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
4,10194018,"[wall, floor;flooring, ceiling, person;individ...","[[44, 44, 6, 6, 6, 6, 6, 6, 6, 6, 6, 44, 44, 4..."
...,...,...,...
1054,33786665,"[wall, floor;flooring, painting;picture, poste...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1055,34103441,"[wall, floor;flooring, person;individual;someo...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1056,34143902,"[wall, person;individual;someone;somebody;mort...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1057,34143939,"[wall, person;individual;someone;somebody;mort...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


In [80]:
# Сохраняем дамп мертаданных ade20k
with open(mypath(root_dir,r'df_ade20k_metadata.pickle'), 'wb') as f:
    pickle.dump(df_ade20k_metadata, f)

In [ ]:
# Модель семантической сегментации Xception_pascalvoc __________________________________________________________________________________

In [16]:
# загружаем таблицу цветовой маркировки pascalvoc в датафрейм pandas
labels_table_pascalvoc = mypath(maps_dir,r'pascalvoc_semantic_segmentation_classes.txt')
df_pascalvoc_labels = pd.read_csv(mypath(root_dir,labels_table_pascalvoc), sep='\t')

In [18]:
# загружаем в модель таблицу весов
segment_image.load_pascalvoc_model(mypath(weights_dir,'deeplabv3_xception_tf_dim_ordering_tf_kernels (2).h5'))

In [92]:
# Тестируем работоспособность 
test_file = r'10194039.1.jpg'
segvalues, output  = segment_image.segmentAsPascalvoc(mypath(test_dir,test_file),overlay = True)
cv2.imwrite(mypath(test_out_dir,test_file[:-4])+'.segm.jpg', output)
labels = text_labels(list(np.unique(segvalues)), df_pascalvoc_labels)
print(labels)
print(output.shape)
print(str(mypath(test_out_dir,test_file[:-4])+'.segm.jpg'))

[" 'background '", "'cow'", "'horse'", "'person'"]
(609, 800, 3)
G:\bulla\test_out\10194039.1.segm.jpg


In [94]:
# Запускаем цикл Xception pascalvoc для основного набора данных. Получаем зображения с наложением маски и датафрейм  разметкой сущностей.
model_dir =pascalvoc_dir
df_pascalvoc_metadata = pd.DataFrame(columns=['ID', 'Things', 'Mask'])
for dirpath, dirnames, filenames in os.walk(file_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                segvalues, output  = segment_image.segmentAsPascalvoc(mypath(file_dir,x),overlay = True)
                cv2.imwrite(mypath(out_dir,model_dir,x[:-4])+'.segm.jpg', output)
                labels = text_labels(list(np.unique(segvalues)), df_pascalvoc_labels)
                row = {'ID':x[:-6], 'Things':labels, 'Mask':segvalues}
                df1 = pd.DataFrame([row])
                df_pascalvoc_metadata = pd.concat([df_pascalvoc_metadata,df1],axis = 0,ignore_index=True)

In [106]:
# Сохраняем дамп датафрейма метаданных
with open(mypath(root_dir,r'df_pascalvoc_metadata.pickle'), 'wb') as f:
    pickle.dump(df_pascalvoc_metadata, f)

In [21]:
with open(mypath(r'G:\bulla\main_data\semantic_df_full',r'df_pascalvoc_metadata.pickle'), 'rb') as f:
    df_pascalvoc_metadata = pickle.load(f)

In [107]:
df_pascalvoc_metadata

,ID,Things,Mask
0,10193973,"[ 'background ', 'person']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,10193980,"[ 'background ', 'person', 'train']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,10193983,"[ 'background ', 'person', 'train']","[[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, ..."
3,10194012,"[ 'background ', 'person', 'train']","[[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, ..."
4,10194018,"[ 'background ', 'person', 'train']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
1054,33786665,[ 'background '],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1055,34103441,"[ 'background ', 'chair', 'person']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1056,34143902,"[ 'background ', 'person']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1057,34143939,"[ 'background ', 'person']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [9]:
# Тест Mask-R-CNN
segment_instance=instance_segmentation() # infer_speed = "average"  - выбор алгоритма скорости. Убрать параметр для выбора значени по умолчанию ( медленно)
segment_instance.load_model(mypath(weights_dir,'mask_rcnn_coco.h5'))


In [7]:
labels_table_mrcnn = mypath(maps_dir,r'mrcnn_labels.txt')
df_mrcnn_labels = pd.read_csv(mypath(root_dir,labels_table_mrcnn), sep='\t')

In [8]:
df_mrcnn_labels

,Idx,Name
0,1,person
1,2,bicycle
2,3,car
3,4,motorcycle
4,5,airplane
...,...,...
75,76,vase
76,77,scissors
77,78,teddy bear
78,79,hair drier


In [32]:
# Запускаем цикл Mask-R-CNN COCO для основного набора данных. С параметром average. Получаем зображения с наложением маски и датафрейм  разметкой сущностей.
model_dir = mrcnn_average_dir
df_mrcnn_average_metadata = pd.DataFrame(columns=['ID', 'Things', 'Mask','ROI'])
for dirpath, dirnames, filenames in os.walk(file_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                segmask, output = segment_instance.segmentImage(mypath(file_dir,x), show_bboxes= True)
                cv2.imwrite(mypath(out_dir, model_dir,x[:-4])+'.instance_overlay_boxed.jpg', output)
                labels = segmask['class_ids'] 
                labels = text_labels(labels, df_mrcnn_labels)
                row = {'ID':x[:-6], 'Things':labels, 'Mask':segmask['masks'],'ROI':segmask['rois']}
                df1 = pd.DataFrame([row])
                df_mrcnn_average_metadata = pd.concat([df_mrcnn_average_metadata,df1],axis = 0,ignore_index=True)

In [33]:
# Сохраняем дамп датафрейма метаданных
with open(mypath(root_dir,r'df_mrcnn_average_metadata'), 'wb') as f:
    pickle.dump(df_mrcnn_average_metadata, f)

In [10]:
# Запускаем цикл Mask-R-CNN COCO для основного набора данных. Получаем зображения с наложением маски и датафрейм  разметкой сущностей.
model_dir = mrcnn_dir
df_mrcnn_metadata = pd.DataFrame(columns=['ID', 'Things', 'Mask','ROI'])
for dirpath, dirnames, filenames in os.walk(file_dir):
        for x in filenames:
            if x[-3:] == 'jpg':
                segmask, output = segment_instance.segmentImage(mypath(file_dir,x), show_bboxes= True)
                cv2.imwrite(mypath(out_dir, model_dir,x[:-4])+'.instance_overlay_boxed.jpg', output)
                labels = segmask['class_ids'] 
                labels = text_labels(labels, df_mrcnn_labels)
                row = {'ID':x[:-6], 'Things':labels, 'Mask':segmask['masks'],'ROI':segmask['rois']}
                df1 = pd.DataFrame([row])
                df_mrcnn_metadata = pd.concat([df_mrcnn_metadata,df1],axis = 0,ignore_index=True)

C:\Python36\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [11]:
# Сохраняем дамп датафрейма метаданных
with open(mypath(root_dir,r'df_mrcnn_metadata.pickle'), 'wb') as f:
    pickle.dump(df_mrcnn_metadata, f)